---

<div align="center">

# Particle Swarm Optimization
</div>

---

In [1]:
import random as rnd
import numpy as np
import matplotlib.pyplot as plt

In [16]:
config = {
    'W':0.5,
    'c1':0.8,
    'c2':0.9,
    'NumIterations':50,
    'NumParticles':30, # Number of Particles to include within the search space
    'targetError':1e-6, # Minimum error that must be satisfied before stopping the search
    'plotIntervals':100
}

In [5]:
class Particle:
    def __init__(self) -> None:
        """
        # Description
            -> Constructor method which allows to create new instances of Particle
        := return: A new instance of Particle
        """
        # Initialize a random position
        x = (-1) ** bool(rnd.getrandbits(1)) * rnd.random() * 1000
        y = (-1) ** bool(rnd.getrandbits(1)) * rnd.random() * 1000
        self.position = np.array([x, y])

        # Initialize the particle's velocity [Static at first]
        self.velocity = np.array([0, 0])

        # Define the personal best position and value
        self.personalBestPosition = self.position
        self.personalBestValue = float('inf')

    def update(self) -> None:
        """
        # Description:
            -> The update method allows the particle to move by adding the velocity to the current position
        := return: None, since we are only updating the particle's position
        """
        # Update current position by adding the veocity
        self.position = self.position + self.velocity

In [13]:
class Space():
    def __init__(self, target:int, config:dict) -> None:
        """
        # Description
            -> Constructor method which allows to create new instances of Space
        := param: target - Target Position aka common goal (The place where all the particles must converge to)
        := param: config - Configuration to be used during the particles search
        := return: A new instance of Space
        """
        # Store all the given parameters
        self.target = target
        self.config = config

        # Create a list to store all the Particles
        self.particles = [Particle() for _ in range(self.config['NumParticles'])]

        # Define the initial global best position and value
        self.globalBestValue = float('inf')
        self.globalBestPosition = np.array([rnd.random() * 50, rnd.random() * 50])

    def fitness(self, particle:Particle) -> int:
        """
        # Description
            -> The fitness method is responsible for performing the fitness score evaluation
        := param: particle - Instance of the previously implemented Particle class
        := return: Fitness score of the given particle
        """
        x = particle.position[0]
        y = particle.position[1]
        f = x**2 + y**2 + 1
        return f

    def setPersonalBest(self) -> None:
        """
        # Description
            -> This method is responsible for Updating the particles personal best position and value if any was found
        := return: None, since it is updating the particles attributes
        """
        # Loop over the Particles
        for particle in self.particles:
            # Calculate the fitness score of the current particle
            fitnessScore = self.fitness(particle)

            # Found a particle that lead to a better result (lead to a better fitnessScore) 
            if (particle.personalBestValue > fitnessScore):
                # Update personal best value and position
                particle.personalBestValue = fitnessScore
                particle.personalBestPosition = particle.position

    def setGlobalBest(self) -> None:
        """
        # Description:
            -> The setGlobalBest method updates the global best value and position if any was found
        := return: None, since it is only updating values
        """
        # Loop over the particles
        for particle in self.particles:
            # Calculate current particle fitness score
            bestFitnessScore = self.fitness(particle)

            # Check if the current fitness score is better than the global one 
            if (self.globalBestValue > bestFitnessScore):
                # Update the Global best variables
                self.globalBestValue = bestFitnessScore
                self.globalBestPosition = particle.position

    def updateParticles(self) -> None:
        """
        # Description
            -> This method updates the particle's position according to a new calculated velocity
        := return: None, since it is only updating the particles position and velocity
        """
        # Loop over the Particles
        for particle in self.particles:
            # Calculate the inertia according to the particle's velocity
            inertia = self.config['W'] * particle.velocity

            # Calculate self and swarm confidences
            selfConfidence = self.config['c1'] * rnd.random() * (particle.personalBestPosition - particle.position)
            swarmConfidence = self.config['c2'] * rnd.random() * (self.globalBestPosition - particle.position)

            # Get a new velocity
            newVelocity = inertia + selfConfidence + swarmConfidence

            # Update the particle's velocity and position
            particle.velocity = newVelocity
            particle.update()

    def showParticles(self, iteration:int) -> None:
        """
        # Description
            -> The showParticles method creates a plot to visualize the particles position within a given iteration
        := return: None, since it is only plotting the particles data
        """
        # Print current iteration and the corresponding global values
        print(f"{iteration} iteration(s)")
        print(f"[Current Best Position] : {self.globalBestPosition}")
        print(f"[Current Best Value / Fitness Score] : {self.globalBestValue}")

        # Loop over the particles and add their data into the plot
        for particle in self.particles:
            plt.plot(particle.position[0], particle.position[1], 'ro')
        plt.plot(self.globalBestPosition[0], self.globalBestPosition[1], 'bo')
        plt.show()

    def performSearch(self) -> None:
        """
        # Description
            -> This method is responsible for performing search with the particles towards the goal
        := param: NumIterations - Number of iterations to be performed
        := return: None, due to the fact that we are merely performing search with the particles towards a common goal
        """
        # Perform <NumInterations> iterations
        for iteration in range(self.config['NumIterations']):
            # Set particles best and global best 
            self.setPersonalBest()
            self.setGlobalBest()

            # Plot the current iteration
            if ((iteration + 1) % self.config['plotIntervals'] == 0):
                self.showParticles(iteration)

            # Check if the current error is small enough and therefore stop the iterations
            if (abs(self.globalBestValue - self.target) <= self.config['targetError']):
                print(f"[Global Best Position] : {self.globalBestPosition} -> Found in {iteration} iteration(s)")
                return

            # Update particles
            self.updateParticles()

        # Print the best solution
        print(f"[Global Best Position] : {self.globalBestPosition} -> Found in {NumIterations} iteration(s)")

In [14]:
searchSpace = Space(1, targetError, NumParticles)
searchSpace.performSearch(NumIterations)

[Global Best Position] : [0.00034689 0.00035197] -> Found in 29 iteration(s)
